# Calculate GWP and make tabels for the paper...

Description of the experiments. bla bla

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '{:,.3f}'.format(x) if abs(x)<0 else ('{:,.2f}'.format(x) if abs(x)<10 else ('{:,.1f}'.format(x) if abs(x)<100 else '{:,.0f}'.format(x))))

Input and output path:

In [2]:
path = r"./input/"
outputpath= r"./output/"

Constants:

In [3]:
#AGWP100_CO2 [mW yr m-2 Tg-1] Source: REF.
agwp100_CO2 = 0.09170

#CH4 tau_strat[yr] Source: REF
tau_strat = 120.0

#CH4 tau_soil [yr] Source: REF
tau_soil = 160.0

#Specific RF for CH4 [mW m-2 ppb-1] Etminan et al., 2016
spec_rf_ch4 = 0.44300

As there is large uncertanty in dry deposition values, we will, in addition to using the dry deposition calculated by the model (but not used in the model, as the surface concentration is fixed), set the dry deposition in the control run to a high and a low value based on literature (IS 80 and 50 OK?). This will result in a range of GWP values. Adjust the perturbations with the same relative factors as in the control.

In [4]:
adjust_drydep = False
if(adjust_drydep):
    drydep = 50.0
    outputpath = outputpath + 'drydep_'+ f'{drydep:.0f}_'

## Read input data

To run this notebook, preprosessed output from the 3 set of simulations are put into a folder named input.
- CTRL: Control simulation, fixed surface concentration
- 10H2: Hydrogen perturbation, surface H_2 increased by 10%. Run to steady state.
- 10CH4: Methane perturbation, surface CH_4 increased by 10%. Run to steady state.

In addition, emission driven runs are added. GFDL-emi: description of this experiment. Or just cite article?

Add table with length of simulation and over how many years the model results are averaged. Data in input is based on monthly data for this averaged data.

## Hydrogen results

### H2 burden [Tg]:

In [5]:
file = 'H2_burden.txt'
df_h2_burden = pd.read_csv(path + file, sep=';',index_col=0,header=0)
delta = df_h2_burden.loc['10H2']-df_h2_burden.loc['CTRL']
delta.name = 'deltaH2'
df_h2_burden = df_h2_burden.append(delta)
df_h2_burden

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
CTRL,196,195,194,192,209
10H2,215,213,213,656,236
10CH4,197,195,195,nan,nan
deltaH2,18.7,18.7,18.6,464,27.3


### H2 loss
Hydrogen loss happens through two main processes. The largest loss is through dry deposition at the ground. Remaining hydrogen is lost through reactions with OH as it ascends through the atmosphere. 

#### H2 drydep [Tg/yr]

The models diagnose drydeposition based on their own schemes, but this is not used directly in the concentration driven runs.

In [6]:
file = 'H2_drydep.txt'
df_h2_drydep = pd.read_csv(path + file, sep=';',index_col=0,header=0)
df_h2_drydep

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
CTRL,59.5,73.0,52.1,54.7,58.3
10H2,65.5,80.3,57.2,204,66.9
10CH4,59.5,73.0,52.1,nan,0.00


In [7]:
if(adjust_drydep):
    print('NB drydep adjusted')
    
    #Adjust by the relative adjustment in the control simulations
    adjust = drydep/df_h2_drydep.loc['CTRL']
    df_h2_drydep = df_h2_drydep*adjust
    
    print(df_h2_drydep)

#### H2 atm. loss [Tg/yr]

In [8]:
file = 'H2_atm_loss.txt'
df_h2_atmloss = pd.read_csv(path + file, sep=';',index_col=0,header=0)
df_h2_atmloss


,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
CTRL,28.4,29.0,22.1,22.1,30.1
10H2,31.0,31.7,24.2,69.4,33.9
10CH4,27.6,28.2,21.6,nan,0.00


#### H2 total loss [Tg/yr]:

In [9]:
df_h2_loss = df_h2_atmloss + df_h2_drydep
df_h2_loss


,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
CTRL,87.9,102,74.2,76.8,88.4
10H2,96.4,112,81.4,273,101
10CH4,87.2,101,73.7,nan,0.00


### H2 production

#### H2 atm. prod [Tg/yr]

In [10]:
file = 'H2_atm_prod.txt'
df_h2_atmprod = pd.read_csv(path + file, sep=';',index_col=0,header=0)
df_h2_atmprod

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
CTRL,56.3,33.9,46.7,44.7,56.2
10H2,56.2,33.8,46.6,43.1,56.1
10CH4,59.2,36.4,48.9,nan,0.00


### Estimated emissions (Total loss = Total prod + emission)

In surface concentration driven runs, there are two unknowns, emissions and dry deposition. 
Emission driven runs, are driven by emission estimates and use the dry deposition scheme to calculate the concentration at the surface. Althogh the emission driven runs use dry deposition and emissions, we adjust the dry deposition as well to take into account uncertainties in emissions and dry deposition on the GWP values.

In [11]:
df_h2_estemis = df_h2_atmloss.loc['CTRL'] + df_h2_drydep.loc['CTRL'] - df_h2_atmprod.loc['CTRL']
df_h2_estemis

OSLOCTM3       31.6
WACCM6-2deg    68.1
INCA           27.5
GFDL-emi       32.1
OSLOCTM3-emi   32.2
Name: CTRL, dtype: float64

### H2 surface concentration [ppb]

In [12]:
file = 'H2_surfconc.txt'
df_h2_surfconc = pd.read_csv(path + file, sep=';',index_col=0,header=0)
delta = df_h2_surfconc.loc['10H2']-df_h2_surfconc.loc['CTRL']
delta.name = 'deltaH2'
df_h2_surfconc = df_h2_surfconc.append(delta)
df_h2_surfconc

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
CTRL,532,532,532,534,567
10H2,585,585,585,"1,880",647
deltaH2,53.2,53.2,53.2,"1,346",80.2


### H2 lifetime [yr]
We calculate the lifetime as burden divided by loss.

In [13]:
df_h2_lifetime = df_h2_burden.drop('deltaH2')/df_h2_loss
df_h2_lifetime

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
CTRL,2.23,1.91,2.62,2.50,2.36
10H2,2.23,1.91,2.62,2.40,2.34
10CH4,2.26,1.93,2.64,nan,nan


### H2 atmospheric lifetime [yr]
The atmospheric lifetime is the burden divided only by the atmospheric loss. This is the lifetime of the fraction of hydrogen which is not dry deposited.

In [14]:
df_h2_atm_lifetime = df_h2_burden.drop('deltaH2')/df_h2_atmloss
df_h2_atm_lifetime

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
CTRL,6.91,6.70,8.79,8.66,6.95
10H2,6.94,6.73,8.82,9.44,6.97
10CH4,7.12,6.93,9.01,nan,nan


### H2 flux (burden dived by lifetime) [Tg/yr]

Add a description of the delta flux here. Include feedback

In [15]:
df_h2_flux = df_h2_burden.drop('deltaH2')/df_h2_lifetime
#Add delta flux 10H2:
delta = df_h2_flux.loc['10H2']-df_h2_flux.loc['CTRL']
delta.name = 'deltaH2'
df_h2_flux = df_h2_flux.append(delta)
#Add delta flux 10CH4:
delta = df_h2_flux.loc['10CH4']-df_h2_flux.loc['CTRL']
delta.name = 'deltaCH4'
df_h2_flux = df_h2_flux.append(delta)
df_h2_flux


,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
CTRL,87.9,102,74.2,76.8,88.4
10H2,96.4,112,81.4,273,101
10CH4,87.2,101,73.7,nan,nan
deltaH2,8.54,9.93,7.19,196,12.4
deltaCH4,-0.75,-0.86,-0.49,nan,nan


Description of the negative deltaCH4. In the methane run, the hydrogen surface concentration is kept fixed. Enhancing methane would influence H2. The hydrogen concentration would have increased, but since we run with fixed concentration, there is a negative flux to compensate. So the increased flux in H2 due to methane is -1*deltaCH4.

## Methane results

### CH4 burden [Tg]

In [16]:
file = 'CH4_burden.txt'
df_ch4_burden = pd.read_csv(path + file, sep=';',index_col=0,header=0)
delta = df_ch4_burden.loc['10CH4']-df_ch4_burden.loc['CTRL']
delta.name = 'deltaCH4'
df_ch4_burden = df_ch4_burden.append(delta)
df_ch4_burden

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
CTRL,"4,995","5,003","4,994","4,935","4,995"
10H2,"4,995","5,003","4,995","4,935","4,995"
10CH4,"5,496","5,505","5,494",nan,nan
deltaCH4,501,502,500,nan,nan


### CH4 atmospheric loss [Tg/yr]

In [17]:
file = 'CH4_loss.txt'
df_ch4_loss = pd.read_csv(path + file, sep=';',index_col=0,header=0)
df_ch4_loss

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
CTRL,683,727,550,582,681
10H2,680,724,547,537,677
10CH4,728,772,588,nan,nan


### CH4 surface concentration [ppb]

In [18]:
file = 'CH4_surfconc.txt'
df_ch4_surfconc = pd.read_csv(path + file, sep=';',index_col=0,header=0)
delta = df_ch4_surfconc.loc['10CH4']-df_ch4_surfconc.loc['CTRL']
delta.name = 'deltaCH4'
df_ch4_surfconc = df_ch4_surfconc.append(delta)
df_ch4_surfconc

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
CTRL,"1,813","1,808","1,813","1,804","1,813"
10CH4,"1,994","1,989","1,994",nan,nan
deltaCH4,181,181,181,nan,nan


### CH4 lifetime [yr]

In [19]:
df_ch4_lifetime = df_ch4_burden.drop('deltaCH4')/df_ch4_loss
df_ch4_lifetime

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
CTRL,7.31,6.88,9.09,8.48,7.33
10H2,7.34,6.91,9.12,9.19,7.38
10CH4,7.55,7.13,9.35,nan,nan


### Total CH4 lifetime [yr]

In [20]:
df_ch4_tot_lifetime = 1.0/(1.0/df_ch4_lifetime + 1.0/tau_strat + 1.0/tau_soil)
df_ch4_tot_lifetime

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
CTRL,6.61,6.25,8.02,7.54,6.62
10H2,6.63,6.28,8.05,8.10,6.66
10CH4,6.80,6.46,8.23,nan,nan


### CH4 flux (burden dived by total lifetime) [Tg/yr]

In [21]:
df_ch4_flux = df_ch4_burden.drop('deltaCH4')/df_ch4_tot_lifetime
#Add delta CH4 flux 10H2
delta = df_ch4_flux.loc['10H2']-df_ch4_flux.loc['CTRL']
delta.name = 'deltaH2'
df_ch4_flux = df_ch4_flux.append(delta)
#Add delta CH4 flux 10CH4
delta = df_ch4_flux.loc['10CH4']-df_ch4_flux.loc['CTRL']
delta.name = 'deltaCH4'
df_ch4_flux = df_ch4_flux.append(delta)
df_ch4_flux


,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
CTRL,756,800,622,654,754
10H2,753,797,620,609,750
10CH4,808,852,668,nan,nan
deltaH2,-2.67,-3.28,-2.10,-45.2,-3.99
deltaCH4,52.4,52.4,45.3,nan,nan


## Ozone results

### Ozone delta DU troposphere

The tropopause definition is the model layer in the control simulation where 150 ppbv ozone are reached. For GFDL-emi 10H2 is H2 perturbation and 10CH4 is H2+CH4 pert minus the H2 pert.

In [22]:
file = 'ozone_du_trop.txt'
df_ozone_du_trop = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_ozone_du_trop

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
10H2,0.04,0.03,0.03,0.73,0.06
10CH4,0.83,0.88,0.54,0.76,nan


### Ozone delta DU stratosphere

In [23]:
file = 'ozone_du_strat.txt'
df_ozone_du_strat = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_ozone_du_strat

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
10H2,-0.03,0.00,-0.03,-0.23,0.02
10CH4,0.83,0.83,0.35,1.07,nan


### Stratospheric ozone 60S-60N

In [24]:
file = 'ozone_du_strat_6060.txt'
df_ozone_du_strat_6060 = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_ozone_du_strat_6060

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
10H2,0.00,0.00,-0.02,-0.22,nan
10CH4,0.62,0.66,0.28,0.68,nan


### Stratospheric ozone 90S-60S and 60N-90N

In [25]:
file = 'ozone_du_strat_gt60.txt'
df_ozone_du_strat_gt60 = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_ozone_du_strat_gt60

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
10H2,0.00,0.00,-0.01,-0.01,nan
10CH4,0.23,0.17,0.08,0.40,nan


### Ozone RF

Ozone RF is calculated using a radiative kernel (Skeie et al 2020) and the modelled changes in ozone concentration. For GFDL-emi 10H2 RF is forcing calculated in the H2 perturbation and 10CH4 the forcing calculated by the H2+CH4 pert minus the H2 pert.

In [26]:
file = 'ozone_rf.txt'
df_ozone_rf = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_ozone_rf

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
10H2,1.86,1.73,1.12,34.0,2.75
10CH4,41.0,42.9,22.6,37.6,nan


In [27]:
df_ozone_rf['OSLOCTM3-emi'].loc['10CH4'] = df_ozone_rf['OSLOCTM3'].loc['10CH4'] 
df_ozone_rf

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
10H2,1.86,1.73,1.12,34.0,2.75
10CH4,41.0,42.9,22.6,37.6,41.0


### Stratospheric H2O RF [mW m-2]

Stratospheric H2O RF calculated offline. Full calculations done for OsloCTM3. Will be done for the other models. For the other models: So far LW instantaneous RF multiply by 0.85 as an assumption to acount for SW and LW adjustments.

In [28]:
file = 'H2O_rf.txt'
df_h2o_rf = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_h2o_rf

,OSLOCTM3,WACCM6-2deg,INCA,OSLOCTM3-emi
Scenario,,,,
10H2,1.48,0.24,0.40,nan
10CH4,10.1,3.00,3.40,nan


In [29]:
df_h2o_rf['OSLOCTM3-emi'].loc['10CH4'] = df_h2o_rf['OSLOCTM3'].loc['10CH4'] 
df_h2o_rf

,OSLOCTM3,WACCM6-2deg,INCA,OSLOCTM3-emi
Scenario,,,,
10H2,1.48,0.24,0.40,nan
10CH4,10.1,3.00,3.40,10.1


In [30]:
file = 'H2O_rf_gfdl.txt'
h2o_rf_gfdl = pd.read_csv(path + file, sep=',',index_col=0,header=0)
h2o_rf_gfdl

,GFDL
H2 [mW m-2],28.3
H2+CH4 [mW m-2],53.1



# Start calculations:

ADD DESCRIPTION OF GWP CALCULATION steady-state = integral of a pulse. Here we can cite :cite:p:`Prather2002a` and :cite:p:`Prather2007a`


### Change in H2 surface conc. caused by 1 Tg H2/yr [ppb yr Tg-1]

This is not used for the GWP calculation. Only for the per flux table and for the feedback factor calulations.

In [31]:
df_surf_h2_per_h2_flux = df_h2_surfconc.loc['deltaH2']/df_h2_flux.loc['deltaH2']
df_surf_h2_per_h2_flux.name = 'surf_h2_per_h2_flux'
df_surf_h2_per_h2_flux

OSLOCTM3       6.23
WACCM6-2deg    5.36
INCA           7.39
GFDL-emi       6.86
OSLOCTM3-emi   6.47
Name: surf_h2_per_h2_flux, dtype: float64

### Change in CH4 flux caused by 1 TgH2 /yr (includes H2 feedback) [Tg CH4/Tg H2]:

The ch4_flux is multiplied by -1 (see above, or move description here)... Why this includes feedback...

In [32]:
df_ch4_flux_per_h2_flux = -1.0*df_ch4_flux.loc['deltaH2']/df_h2_flux.loc['deltaH2']
df_ch4_flux_per_h2_flux.name = 'ch4_flux_per_h2_flux'
df_ch4_flux_per_h2_flux

OSLOCTM3       0.31
WACCM6-2deg    0.33
INCA           0.29
GFDL-emi       0.23
OSLOCTM3-emi   0.32
Name: ch4_flux_per_h2_flux, dtype: float64

### Change in CH4 surface conc. caused by 1 Tg/yr CH4 [ppb yr/Tg CH4]

In [33]:
df_surf_ch4_per_ch4_flux =  df_ch4_surfconc.loc['deltaCH4']/df_ch4_flux.loc['deltaCH4']
df_surf_ch4_per_ch4_flux.name = 'surf_ch4_per_ch4_flux'
df_surf_ch4_per_ch4_flux

OSLOCTM3       3.46
WACCM6-2deg    3.45
INCA           4.00
GFDL-emi        nan
OSLOCTM3-emi    nan
Name: surf_ch4_per_ch4_flux, dtype: float64

Set equal to OsloCTM3 concentrationdriven

In [34]:
df_surf_ch4_per_ch4_flux['OSLOCTM3-emi'] = df_surf_ch4_per_ch4_flux['OSLOCTM3']
df_surf_ch4_per_ch4_flux

OSLOCTM3       3.46
WACCM6-2deg    3.45
INCA           4.00
GFDL-emi        nan
OSLOCTM3-emi   3.46
Name: surf_ch4_per_ch4_flux, dtype: float64

### Change in CH4 surface concentration per emission H2 [ppb yr /Tg H2]

In [35]:
df_surf_ch4_per_h2_flux = df_surf_ch4_per_ch4_flux*df_ch4_flux_per_h2_flux
df_surf_ch4_per_h2_flux.name = 'surf_ch4_per_h2_flux'

#Add GFDL:
#Increase in surface concentration CH4:
#1808 to 2005 ppbv (REF Paolot)
df_surf_ch4_per_h2_flux['GFDL-emi']= (2005.-1808.)/df_h2_flux['GFDL-emi'].loc['deltaH2']
df_surf_ch4_per_h2_flux

OSLOCTM3       1.08
WACCM6-2deg    1.14
INCA           1.17
GFDL-emi       1.00
OSLOCTM3-emi   1.11
Name: surf_ch4_per_h2_flux, dtype: float64

### Change in H2 flux caused by 1 TgCH4/yr [Tg H2/Tg CH4]

Why we multiply by -1

In [36]:
df_h2_flux_per_ch4_flux = -1.0*df_h2_flux.loc['deltaCH4']/df_ch4_flux.loc['deltaCH4']
df_h2_flux_per_ch4_flux.name = 'h2_flux_per_ch4_flux'
df_h2_flux_per_ch4_flux

OSLOCTM3       0.01
WACCM6-2deg    0.02
INCA           0.01
GFDL-emi        nan
OSLOCTM3-emi    nan
Name: h2_flux_per_ch4_flux, dtype: float64

In [37]:
df_h2_flux_per_ch4_flux['OSLOCTM3-emi'] = df_h2_flux_per_ch4_flux['OSLOCTM3']
df_h2_flux_per_ch4_flux

OSLOCTM3       0.01
WACCM6-2deg    0.02
INCA           0.01
GFDL-emi        nan
OSLOCTM3-emi   0.01
Name: h2_flux_per_ch4_flux, dtype: float64

### HYDROGEN AGWP100 CH4 [mW m-2 yr Tg-1]

Explain why this is AGWP

In [38]:
df_h2_agwp_ch4 = df_surf_ch4_per_h2_flux*spec_rf_ch4
df_h2_agwp_ch4.name = 'h2_agwp_ch4'


#agwp_ch4 = RF per flux H2 (For the per flux table)
df_ch4_rf_per_h2_flux = df_surf_ch4_per_h2_flux*spec_rf_ch4
df_ch4_rf_per_h2_flux.name = 'ch4_rf_per_h2_flux'





### Initialize H2 GWP table

In [39]:
antmod = len(df_h2_agwp_ch4.index)
df_h2_gwp = pd.DataFrame(np.empty([5,antmod])*np.nan,columns=df_h2_agwp_ch4.index,
                         index=['O3','CH4','strat H2O','O3 CH4ind','strat H2O CH4ind'])

### Add methane GWP

In [40]:
df_h2_gwp.loc['CH4'] = df_h2_agwp_ch4/agwp100_CO2

### HYDROGEN AGWP100 strat H2O [mW m-2 yr Tg-1]

Explain that RF/flux equal AGWP..

In [41]:
df_h2_agwp_h2o = df_h2o_rf.loc['10H2']/df_h2_flux.loc['deltaH2']
df_h2_agwp_h2o['GFDL-emi'] = h2o_rf_gfdl['GFDL'].loc['H2+CH4 [mW m-2]']/df_h2_flux['GFDL-emi'].loc['deltaH2']
df_h2_agwp_h2o.name = 'h2_agwp_h2o'

#NBNB GFDL methane induced is included.

In [42]:
#Add to the flux table
df_h2o_rf_per_h2_flux = df_h2o_rf.loc['10H2']/df_h2_flux.loc['deltaH2']
df_h2o_rf_per_h2_flux.name= 'h2o_rf_per_h2_flux'
df_h2o_rf_per_h2_flux['GFDL-emi'] = h2o_rf_gfdl['GFDL'].loc['H2 [mW m-2]']/df_h2_flux['GFDL-emi'].loc['deltaH2']

#Strat H2O RF per methane flux (Move this to the methane part?)
df_h2o_rf_per_ch4_flux = df_h2o_rf.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_h2o_rf_per_ch4_flux.name = 'h2o_rf_per_ch4_flux'


In [43]:
df_h2o_rf_per_ch4_flux['OSLOCTM3-emi'] = df_h2o_rf_per_ch4_flux['OSLOCTM3']
df_h2o_rf_per_ch4_flux

GFDL-emi        nan
INCA           0.08
OSLOCTM3       0.19
OSLOCTM3-emi   0.19
WACCM6-2deg    0.06
Name: h2o_rf_per_ch4_flux, dtype: float64

### Add stratospheric H2O GWP

In [44]:
df_h2_gwp.loc['strat H2O'] = df_h2_agwp_h2o/agwp100_CO2

### HYDROGEN AGWP100 O3 [mW m-2 yr Tg-1]

In [45]:
df_h2_agwp_o3 = df_ozone_rf.loc['10H2']/df_h2_flux.loc['deltaH2']
df_h2_agwp_o3.name = 'h2_agwp_o3'

#For GFDL-emi include the methane induced part.
df_h2_agwp_o3['GFDL-emi'] = (df_ozone_rf['GFDL-emi'].loc['10H2']+df_ozone_rf['GFDL-emi'].loc['10CH4'])/df_h2_flux['GFDL-emi'].loc['deltaH2']


In [46]:
#Similar, but use only the H2 Ozone RF for GFDL. To be used in the table:
df_ozone_rf_per_h2_flux = df_ozone_rf.loc['10H2']/df_h2_flux.loc['deltaH2']
df_ozone_rf_per_h2_flux.name= 'ozone_rf_per_h2_flux'

In [47]:
#Ozone RF per methane flux (move to the methane part?)
df_ozone_rf_per_ch4_flux = df_ozone_rf.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_ozone_rf_per_ch4_flux.name = 'ozone_rf_per_ch4_flux'

In [48]:
df_ozone_rf_per_ch4_flux['OSLOCTM3-emi'] = df_ozone_rf_per_ch4_flux['OSLOCTM3']
df_ozone_rf_per_ch4_flux

OSLOCTM3       0.78
WACCM6-2deg    0.82
INCA           0.50
GFDL-emi        nan
OSLOCTM3-emi   0.78
Name: ozone_rf_per_ch4_flux, dtype: float64

### Add Ozone GWP

In [49]:
df_h2_gwp.loc['O3'] = df_h2_agwp_o3/agwp100_CO2

### For the per flux table

In [50]:
df_trop_du_ozone_per_ch4_flux = df_ozone_du_trop.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_trop_du_ozone_per_ch4_flux.name = 'trop_du_ozone_per_ch4_flux'

df_strat_du_ozone_per_ch4_flux = df_ozone_du_strat.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_strat_du_ozone_per_ch4_flux.name = 'strat_du_ozone_per_ch4_flux'

df_trop_du_ozone_per_h2_flux = df_ozone_du_trop.loc['10H2']/df_h2_flux.loc['deltaH2']
df_trop_du_ozone_per_h2_flux.name = 'trop_du_ozone_per_h2_flux'

df_strat_du_ozone_per_h2_flux = df_ozone_du_strat.loc['10H2']/df_h2_flux.loc['deltaH2']
df_strat_du_ozone_per_h2_flux.name = 'strat_du_ozone_per_h2_flux'


In [51]:
df_trop_du_ozone_per_ch4_flux['OSLOCTM3-emi']=df_trop_du_ozone_per_ch4_flux['OSLOCTM3']
df_strat_du_ozone_per_ch4_flux['OSLOCTM3-emi']=df_strat_du_ozone_per_ch4_flux['OSLOCTM3']

In [52]:
df_strat_du_6060_ozone_per_ch4_flux = df_ozone_du_strat_6060.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_strat_du_6060_ozone_per_ch4_flux.name = 'strat_du_6060_ozone_per_ch4_flux'

df_strat_du_6060_ozone_per_h2_flux = df_ozone_du_strat_6060.loc['10H2']/df_h2_flux.loc['deltaH2']
df_strat_du_6060_ozone_per_h2_flux.name = 'strat_du_6060_ozone_per_h2_flux'

df_strat_du_gt60_ozone_per_ch4_flux = df_ozone_du_strat_gt60.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_strat_du_gt60_ozone_per_ch4_flux.name = 'strat_du_gt60_ozone_per_ch4_flux'

df_strat_du_gt60_ozone_per_h2_flux = df_ozone_du_strat_gt60.loc['10H2']/df_h2_flux.loc['deltaH2']
df_strat_du_gt60_ozone_per_h2_flux.name = 'strat_du_gt60_ozone_per_h2_flux'

## Methane induced GWP:

### HYDROGEN AGWP100 methane induced O3 [mW m-2 yr Tg-1]

It does not matter here if we use surface concentration, burden, tropospheric concentration?

In [53]:
df_ch4_surfconc['OSLOCTM3-emi'].loc['deltaCH4']=df_ch4_surfconc['OSLOCTM3'].loc['deltaCH4']

In [54]:
#Wm-2/ppbCH4*ppbCH4/TgH2yr-1 -> Wm-2/TgH2yr-1
df_h2_agwp_ch4ind_o3 = df_ozone_rf.loc['10CH4']/df_ch4_surfconc.loc['deltaCH4']*df_surf_ch4_per_h2_flux
df_h2_agwp_ch4ind_o3.name = 'h2_agwp_ch4ind_o3'


### Add methane induced O3 GWP

In [55]:
df_h2_gwp.loc['O3 CH4ind'] = df_h2_agwp_ch4ind_o3/agwp100_CO2
df_h2_gwp

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
O3,2.38,1.90,1.69,3.98,2.41
CH4,5.22,5.50,5.65,4.85,5.38
strat H2O,1.89,0.26,0.61,2.95,nan
O3 CH4ind,2.67,2.94,1.59,nan,2.75
strat H2O CH4ind,nan,nan,nan,nan,nan


### HYDROGEN AGWP100 methane induced strat H2O [mW m-2 yr Tg-1]

In [56]:
df_h2_agwp_ch4ind_h2o = df_h2o_rf.loc['10CH4']/df_ch4_surfconc.loc['deltaCH4']*df_surf_ch4_per_h2_flux
df_h2_agwp_ch4ind_h2o.name = 'h2_agwp_ch4ind_h2o'

### Add methane induced strat H2O GWP

In [57]:
df_h2_gwp.loc['strat H2O CH4ind'] = df_h2_agwp_ch4ind_h2o/agwp100_CO2
df_h2_gwp

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
O3,2.38,1.90,1.69,3.98,2.41
CH4,5.22,5.50,5.65,4.85,5.38
strat H2O,1.89,0.26,0.61,2.95,nan
O3 CH4ind,2.67,2.94,1.59,nan,2.75
strat H2O CH4ind,0.66,0.21,0.24,nan,0.68


# Methane GWP

Initialize CH4 GWP

In [58]:
antmod = len(df_h2_agwp_ch4.index)
df_ch4_gwp = pd.DataFrame(np.empty([4,antmod])*np.nan,columns=df_h2_agwp_ch4.index,
                         index=['O3','CH4','strat H2O','H2'])
#df_ch4_gwp

### Methane AGWP100 O3 [mW m-2 yr Tg-1]

Integrated O3 from CH4 RF [W m-2 yr Tg-1]

In [59]:
df_ch4_agwp_o3 = df_ozone_rf.loc['10CH4']/df_ch4_surfconc.loc['deltaCH4']*df_surf_ch4_per_ch4_flux
df_ch4_agwp_o3.name = 'ch4_agwp_o3'



### Add ozone GWP

In [60]:
df_ch4_gwp.loc['O3'] =df_ch4_agwp_o3/agwp100_CO2 

### Methane AGWP100 Methane [mW m-2 yr Tg-1]

In [61]:
df_ch4_agwp =df_surf_ch4_per_ch4_flux*spec_rf_ch4
df_ch4_agwp.name = 'ch4_agwp'

### Add methane GWP

In [62]:
#Add Methane GWP:
df_ch4_gwp.loc['CH4'] =df_ch4_agwp/agwp100_CO2 

### Methane AGWP100 strat H2O [mW m-2 yr Tg-1]

In [63]:
print(df_h2o_rf.loc['10CH4'])
df_ch4_agwp_h2o = df_h2o_rf.loc['10CH4']/df_ch4_surfconc.loc['deltaCH4']*df_surf_ch4_per_ch4_flux
df_ch4_agwp_h2o.name = 'ch4_agwp_h2o'

OSLOCTM3       10.1
WACCM6-2deg    3.00
INCA           3.40
OSLOCTM3-emi   10.1
Name: 10CH4, dtype: float64


### Add Strat H2O GWP:

In [64]:
df_ch4_gwp.loc['strat H2O'] = df_ch4_agwp_h2o/agwp100_CO2

### Add GWP via H2

In [65]:
df_ch4_gwp.loc['H2'] = df_h2_flux_per_ch4_flux*df_h2_gwp.sum()

# Main results and tabels

## H2 GWP 100

In [66]:
#Save to file:
#print(outputpath + 'table_h2_gwp.csv')
df_h2_gwp.to_csv(outputpath + 'table_h2_gwp.csv')
df_h2_gwp.loc['total']=df_h2_gwp.sum()
df_h2_gwp

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
O3,2.38,1.90,1.69,3.98,2.41
CH4,5.22,5.50,5.65,4.85,5.38
strat H2O,1.89,0.26,0.61,2.95,nan
O3 CH4ind,2.67,2.94,1.59,nan,2.75
strat H2O CH4ind,0.66,0.21,0.24,nan,0.68
total,12.8,10.8,9.78,11.8,11.2


## CH4 GWP 100

In [67]:
df_ch4_gwp.to_csv(outputpath + 'table_ch4_gwp.csv')
df_ch4_gwp.loc['total']=df_ch4_gwp.sum()
df_ch4_gwp

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
O3,8.55,8.92,5.43,nan,8.55
CH4,16.7,16.7,19.3,nan,16.7
strat H2O,2.10,0.62,0.82,nan,2.10
H2,0.18,0.18,0.11,nan,0.16
total,27.6,26.4,25.7,0.00,27.5


## Table per flux H2

In [68]:
df_per_flux_h2 = pd.concat([df_h2_flux.loc['deltaH2'],df_surf_h2_per_h2_flux,
                            df_surf_ch4_per_h2_flux,df_ch4_flux_per_h2_flux,df_ch4_rf_per_h2_flux,
                            df_trop_du_ozone_per_h2_flux,df_strat_du_ozone_per_h2_flux,
                            df_strat_du_6060_ozone_per_h2_flux,df_strat_du_gt60_ozone_per_h2_flux,
                            df_ozone_rf_per_h2_flux,df_h2o_rf_per_h2_flux],axis=1, sort=False)

#Save to file:
df_per_flux_h2.to_csv(outputpath + 'table_per_flux_h2.csv')

#Rename the columns:
columns_names={'deltaH2':'Flux H2 [Tg/yr]',
               'surf_h2_per_h2_flux': 'Surf. conc. H2 per flux [ppb yr/Tg]',
               'surf_ch4_per_h2_flux':'Surf. conc. CH4 per flux [ppb yr/Tg]',
               'ch4_flux_per_h2_flux':'Flux CH4/Flux H2 [Tg CH4/Tg H2]',
               'ch4_rf_per_h2_flux':'CH4 RF per flux [mW m-2 yr/ Tg]',
               'trop_du_ozone_per_h2_flux':'Trop. ozone per flux [DU yr/Tg]',
               'strat_du_ozone_per_h2_flux':'Strat. ozone per flux [DU yr/Tg]',
               'strat_du_6060_ozone_per_h2_flux':'Strat. ozone 60S-60N per flux [DU yr/Tg]',
               'strat_du_gt60_ozone_per_h2_flux':'Strat. ozone >60S/N per flux [DU yr/Tg]',
               'ozone_rf_per_h2_flux':'ozone RF per flux [mW m-2 yr/ Tg]',
               'h2o_rf_per_h2_flux':'Strat. H2O RF per flux [mW m-2 yr/ Tg]'}
#Rename column names:
df_per_flux_h2.rename(columns=dict(columns_names),inplace=True) #[df_per_flux_h2.columns])
df_per_flux_h2

,Flux H2 [Tg/yr],Surf. conc. H2 per flux [ppb yr/Tg],Surf. conc. CH4 per flux [ppb yr/Tg],Flux CH4/Flux H2 [Tg CH4/Tg H2],CH4 RF per flux [mW m-2 yr/ Tg],Trop. ozone per flux [DU yr/Tg],Strat. ozone per flux [DU yr/Tg],Strat. ozone 60S-60N per flux [DU yr/Tg],Strat. ozone >60S/N per flux [DU yr/Tg],ozone RF per flux [mW m-2 yr/ Tg],Strat. H2O RF per flux [mW m-2 yr/ Tg]
OSLOCTM3,8.54,6.23,1.08,0.31,0.48,0.00,-0.00,0.00,0.00,0.22,0.17
WACCM6-2deg,9.93,5.36,1.14,0.33,0.50,0.00,0.00,0.00,0.00,0.17,0.02
INCA,7.19,7.39,1.17,0.29,0.52,0.00,-0.00,-0.00,-0.00,0.16,0.06
GFDL-emi,196,6.86,1.00,0.23,0.44,0.00,-0.00,-0.00,-0.00,0.17,0.14
OSLOCTM3-emi,12.4,6.47,1.11,0.32,0.49,0.00,0.00,nan,nan,0.22,nan


## Table per flux CH4

To do: fix the table headings in the table (as for H2 above)

In [69]:
df_per_flux_ch4 = pd.concat([df_ch4_flux.loc['deltaCH4'],df_surf_ch4_per_ch4_flux,
                            df_h2_flux_per_ch4_flux,df_trop_du_ozone_per_ch4_flux,df_strat_du_ozone_per_ch4_flux,
                            df_strat_du_6060_ozone_per_ch4_flux,df_strat_du_gt60_ozone_per_ch4_flux,
                            df_ozone_rf_per_ch4_flux,df_h2o_rf_per_ch4_flux],axis=1,sort=False)



               
#Save to file:
df_per_flux_ch4.to_csv(outputpath + 'table_per_flux_ch4.csv')

#Rename the columns:
columns_names={'deltaCH4':'Flux CH4 [Tg/yr]',
               'surf_ch4_per_ch4_flux':'Surf. conc. CH4 per flux [ppb yr/Tg]',
               'h2_flux_per_ch4_flux':'Flux H2/Flux CH4 [Tg H2/Tg CH4]',
               'trop_du_ozone_per_ch4_flux':'Trop. ozone per flux [DU yr/Tg]',
               'strat_du_ozone_per_ch4_flux':'Strat. ozone per flux [DU yr/Tg]',
               'strat_du_6060_ozone_per_ch4_flux':'Strat. ozone 60S-60N per flux [DU yr/Tg]',
               'strat_du_gt60_ozone_per_ch4_flux':'Strat. ozone >60S/N per flux [DU yr/Tg]',
               'ozone_rf_per_ch4_flux':'ozone RF per flux [mW m-2 yr/ Tg]',
               'h2o_rf_per_ch4_flux':'Strat H2O RF per flux [mW m-2 yr/ Tg]'}
               
#Rename column names:
df_per_flux_ch4.rename(columns=dict(columns_names),inplace=True) #[df_per_flux_h2.columns])
df_per_flux_ch4          

,Flux CH4 [Tg/yr],Surf. conc. CH4 per flux [ppb yr/Tg],Flux H2/Flux CH4 [Tg H2/Tg CH4],Trop. ozone per flux [DU yr/Tg],Strat. ozone per flux [DU yr/Tg],Strat. ozone 60S-60N per flux [DU yr/Tg],Strat. ozone >60S/N per flux [DU yr/Tg],ozone RF per flux [mW m-2 yr/ Tg],Strat H2O RF per flux [mW m-2 yr/ Tg]
OSLOCTM3,52.4,3.46,0.01,0.02,0.02,0.01,0.00,0.78,0.19
WACCM6-2deg,52.4,3.45,0.02,0.02,0.02,0.01,0.00,0.82,0.06
INCA,45.3,4.00,0.01,0.01,0.01,0.01,0.00,0.50,0.08
GFDL-emi,nan,nan,nan,nan,nan,nan,nan,nan,nan
OSLOCTM3-emi,nan,3.46,0.01,0.02,0.02,nan,nan,0.78,0.19


## H2 budget tabel

In [70]:
df_budget_h2 = pd.concat([df_h2_burden.loc['CTRL'],df_h2_lifetime.loc['CTRL'],
                          df_h2_atmloss.loc['CTRL'],df_h2_drydep.loc['CTRL'],
                          df_h2_atmprod.loc['CTRL'],df_h2_atm_lifetime.loc['CTRL'],
                          df_h2_estemis],axis=1)

df_budget_h2.columns = ['H2 burden [Tg]',
                        'H2 lifetime [yrs]',
                        'H2 atm.loss[Tg/yr]',
                        'H2 drydep [Tg/yr]',
                        'H2 atm. prod [Tg/yr]',
                        'H2 atmospheric lifetime [yrs]',
                        'H2 estimated emissions [Tg/yr]']

df_budget_h2.to_csv(outputpath + 'table_budget_h2.csv')
df_budget_h2

,H2 burden [Tg],H2 lifetime [yrs],H2 atm.loss[Tg/yr],H2 drydep [Tg/yr],H2 atm. prod [Tg/yr],H2 atmospheric lifetime [yrs],H2 estimated emissions [Tg/yr]
OSLOCTM3,196,2.23,28.4,59.5,56.3,6.91,31.6
WACCM6-2deg,195,1.91,29.0,73.0,33.9,6.70,68.1
INCA,194,2.62,22.1,52.1,46.7,8.79,27.5
GFDL-emi,192,2.50,22.1,54.7,44.7,8.66,32.1
OSLOCTM3-emi,209,2.36,30.1,58.3,56.2,6.95,32.2


## CH4 budget tabel

In [71]:
df_budget_ch4 = pd.concat([df_ch4_burden.loc['CTRL'],df_ch4_lifetime.loc['CTRL'],
                           df_ch4_loss.loc['CTRL']],axis=1)
df_budget_ch4.columns = ['CH4 burden [Tg]','CH4 lifetime [yrs]','CH4 atm.loss[Tg/yr]'] 
df_budget_ch4

,CH4 burden [Tg],CH4 lifetime [yrs],CH4 atm.loss[Tg/yr]
OSLOCTM3,"4,995",7.31,683
WACCM6-2deg,"5,003",6.88,727
INCA,"4,994",9.09,550
GFDL-emi,"4,935",8.48,582
OSLOCTM3-emi,"4,995",7.33,681


# Other results/checks:

## Methane feedback factor:

### Atmospheric mass conversion CH4  [Tg/ppb] (from perturbations)

In [72]:
df_ch4_burden_per_conc  = df_ch4_burden.loc['deltaCH4']/df_ch4_surfconc.loc['deltaCH4']
df_ch4_burden_per_conc.name = 'ch4_burden_per_conc'
df_ch4_burden_per_conc

OSLOCTM3       2.76
WACCM6-2deg    2.78
INCA           2.76
GFDL-emi        nan
OSLOCTM3-emi    nan
Name: ch4_burden_per_conc, dtype: float64

### Increase per unit flux w/o feedback = integrated decay [ppb yr/Tg]

Add explanation... 

In [73]:
df_w_o_feedback =df_ch4_lifetime.loc['CTRL']/df_ch4_burden_per_conc #Lifetime [yr] / [Tg/ppb] 
df_w_o_feedback

OSLOCTM3       2.65
WACCM6-2deg    2.48
INCA           3.30
GFDL-emi        nan
OSLOCTM3-emi    nan
dtype: float64

### Feedback factor: increase CH4 with feedback/ increase CH4 without feedback

In [74]:
df_feedback_factor_ch4 = df_surf_ch4_per_ch4_flux/df_w_o_feedback
df_feedback_factor_ch4.name = 'feedback_factor_ch4'
df_feedback_factor_ch4

OSLOCTM3       1.31
WACCM6-2deg    1.39
INCA           1.21
GFDL-emi        nan
OSLOCTM3-emi    nan
Name: feedback_factor_ch4, dtype: float64

## Hydrogen feedback factor:

### Atmospheric mass conversion H2  [Tg/ppb] (from perturbations)

In [75]:
df_h2_burden_per_conc  = df_h2_burden.loc['deltaH2']/df_h2_surfconc.loc['deltaH2']
df_h2_burden_per_conc.name = 'h2_burden_per_conc'
df_h2_burden_per_conc

OSLOCTM3       0.35
WACCM6-2deg    0.35
INCA           0.35
GFDL-emi       0.34
OSLOCTM3-emi   0.34
Name: h2_burden_per_conc, dtype: float64

### Increase per unit flux w/o feedback = integrated decay [ppb yr/Tg]

In [76]:
df_w_o_feedback_h2 =df_h2_lifetime.loc['CTRL']/df_h2_burden_per_conc #Lifetime [yr] / [Tg/ppb] 
df_w_o_feedback_h2

OSLOCTM3       6.36
WACCM6-2deg    5.44
INCA           7.50
GFDL-emi       7.25
OSLOCTM3-emi   6.95
dtype: float64

### Feedback factor: increase H2 with feedback/ increase H2 without feedback

In [77]:
df_feedback_factor_h2 = df_surf_h2_per_h2_flux/df_w_o_feedback_h2
df_feedback_factor_h2.name = 'feedback_factor_h2'
df_feedback_factor_h2
#Fabien wrote in the paper about feedback factor less than 1.

OSLOCTM3       0.98
WACCM6-2deg    0.99
INCA           0.99
GFDL-emi       0.95
OSLOCTM3-emi   0.93
Name: feedback_factor_h2, dtype: float64